<a href="https://colab.research.google.com/github/rharris9/SYSM578/blob/main/RohanHarrisClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Data and linking the datasets to have data with the desired labels

In [27]:
#import pandas and numpy
import pandas as pd
import numpy as np

#import standard classification tools from sklearn
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import itertools

# Loading SVM problems
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.preprocessing import scale
import warnings

#import tools for ROC curves
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc

#visualization
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

template = 'ggplot2'
test_size = 0.3

data = pd.read_table('https://raw.githubusercontent.com/PineBiotech/omicslogic/master/TCGA_43_clinical.csv',sep='\,',header=(0))

rawdata = pd.read_table('https://raw.githubusercontent.com/PineBiotech/omicslogic/master/LIHC_RSEM_42cases.txt',sep='\t',header=(0))


#prepare data
features = data.iloc[1:, 0].values


X = rawdata.iloc[:, 0:].values
y = data.iloc[:, 1].values

race = np.unique(y)
n_race = len(race)
str(race)

#X = scale(race, axis=1)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=1)


clin_data1 = data.loc[:,('race','Variable')]
clin_data1['Variable'] = clin_data1.loc[:,'Variable'].str.replace('-','_') 
clin_data1

data=data.drop(['Variable'], axis = 1) 
dataT = np.transpose(rawdata)

fdata = pd.merge(dataT, clin_data1, left_index=True, right_on='Variable').drop(columns='Variable')
fdata.head()

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,0,1,2,3,4,5,6,7,8,9,...,20523,20524,20525,20526,20527,20528,20529,20530,race,Variable


First step for Random forest is to train data

In [28]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=0)

#finding out how many samples were from asian patients
print(len(y[y == 'ASIAN']))


ValueError: ignored

In [ ]:

#finding out how many samples were from Black or African American patients
print(len(y[y == 'BLACK OR AFRICAN AMERICAN']))

In [ ]:
#finding out how many samples were from White patients
print(len(y[y == 'WHITE']))

In [ ]:
#Comparing proportion of the study population by race

for sample_type in race:
  print(sample_type, ": ", round(len(y_test[y_test == sample_type])/len(y[y == sample_type]),2))

Random Forest

In [ ]:
#define Random forest model
model = RandomForestClassifier(n_estimators=1000, bootstrap=True, verbose=True)
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

#define Random forest model
model = RandomForestClassifier(n_estimators=1000, bootstrap=True, verbose=False)

#fit model on training data
model.fit(X_train, y_train)

estimator = model.estimators_[10]

#predicting test data
predictions = model.predict(X_test)
print("Model accuracy = ", round(accuracy_score(y_test, predictions),2))


Confusion Matrix for Random Forest classification

In [ ]:
#prepare a confusion matrix
conf = confusion_matrix(y_test,predictions)
new_conf = pd.DataFrame(conf, columns=race, index=race)

import matplotlib.pyplot as plt
import seaborn as sns

#plot heatmap of confusion matrix
fig, ax = plt.subplots(figsize=(13, 10))
sns.heatmap(new_conf, annot=True);

Feature Importance for Random Forest

In [ ]:
importances = model.feature_importances_
std = np.std([tree.feature_importances_ for tree in model.estimators_], axis=0)

forest_importances = pd.Series(importances, index=features)
forest_importances_top = forest_importances[forest_importances > 0.003]
forest_importances_top = forest_importances_top.sort_values()

#plot forest_importances
forest_importances_top.plot.bar(figsize=(15, 10));

SVC Classification

In [ ]:
#ignoring warnings
warnings.filterwarnings('ignore')  # "error", "ignore", "always", "default", "module" or "once"

# Using to find train data
model = LinearSVC(C=1.0)
model.fit(X_train, y_train)

# after training we can predict labels for the test samples
predictions = model.predict(X_test)

c_matrix = confusion_matrix(y_test,predictions)
cr = classification_report(y_test,predictions, output_dict=True)
model_accuracy = accuracy_score(y_test, predictions)

SVC Results

In [ ]:
clsf_report = pd.DataFrame(cr).transpose()
clsf_report1 = clsf_report[0:n_race]

fig = px.bar(clsf_report1, barmode='group', template=template)
fig.update_layout(width=700, height=700, template=template, title="Classification Report")
fig.show()

PCA for SVC Results

In [ ]:
from sklearn.preprocessing import StandardScaler 
from sklearn.decomposition import PCA 

dataPCA = dataT
dataPCA = dataPCA.iloc[:,1:len(dataPCA.columns)]
dataPCA

scaled = StandardScaler() 
scaled.fit(dataPCA) 
scaled_data = scaled.transform(dataPCA) 

#Run PCA
n_components=3
pca = PCA(n_components) 
pca.fit(scaled_data) 
xpca = pca.transform(scaled_data)

labels = {str(i): f'PC{i+1}: {pca.explained_variance_ratio_[i]*100:.2f}%' for i in range(n_components)}
template = 'ggplot2'

# Displaying the PCA for Race
figPCA2D = px.scatter(xpca, x=0, y=1, color=y, 
                      labels=dataPCA.index,
                      hover_name = dataPCA.index,
                      title="PCA for Race")
figPCA2D.update_layout(width=1000, height=800, template=template)
figPCA2D.show()

Confusion Matrix for SVC classification of Race

In [ ]:
CM_df = pd.DataFrame(c_matrix, columns=race, index=race)

heat = go.Heatmap(z = CM_df, x=CM_df.index, 
                  y=CM_df.columns, 
                  showscale = False,
                  text=CM_df,
                  texttemplate="%{text}",
                  textfont={"size":10})
fig = go.Figure(heat)
fig.update_layout(width=700, height=700, template=template, title="Confusion Matrix")
fig.show()

Feature Importance

In [ ]:
clinplain = fdata.drop(columns=['Variable'])
features = fdata.columns
classes = fdata['Variable']


importances = model.feature_importances_
std = np.std([tree.feature_importances_ for tree in model.estimators_], axis=0)

forest_importances = pd.Series(importances, index=features)
forest_importances = forest_importances[forest_importances > 0.0015]
forest_importances = forest_importances.sort_values()

#plot forest_importances
forest_importances.plot.bar(figsize=(15, 10));

In [ ]:
print(classification_report(y_test,predictions, zero_division=0))
print(accuracy_score(y_test, predictions))

In [ ]:
importance = np.abs(model[1].coef_)
feature_names = np.array(features)

result_df = pd.DataFrame(model[1].coef_)
result_df.columns = feature_names 
result_df = result_df.T
result_df.columns = ['coef']
result_df = result_df[result_df['coef'] > 200000]
result_df.plot.bar();

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
#HistGradientBoostingClassifier

# define range of values for each hyperparameter
param_dist = [
    {
        "learning_rate": [0.1, 0.5, 0.8, 0.9],
        "max_iter": [100, 250, 500],
        "max_depth": [1, 10, 100, 1000]
    }
]

#set baseline model
model = HistGradientBoostingClassifier()

#initialise random search with cross-validation
random_search = RandomizedSearchCV(
    model, param_distributions=param_dist, n_iter=40, cv=3, scoring="roc_auc", n_jobs=-1
)
random_search.fit(X_train, y_train)

#fit best model to data
best_model = random_search.best_estimator_
best_model.fit(X_train, y_train)

#print results for test and train data
print("optimal results are: \n", best_model)
print("-----------------------")

model_name1 = str(best_model)[:15]

result = ("Classification Score for TRAIN data using ", model_name1, "... is ", round(best_model.score(X_train, y_train),2))
result = ''.join(str(x) for x in result)
print(result)
result = ("Classification Score for TEST data using ", model_name1, "... is ", round(best_model.score(X_test, y_test),2))
result = ''.join(str(x) for x in result)
print(result)

To increase F1 scores I increased the number of estimators and depth on the random forest with bootstrapping. I was unable to get the PCA for that segment. However with a higher level of depth and would increase the PCA Values